# Práctica 8: Residual neural networks - Parte 2


### Pre-requisitos. Instalar paquetes

Para la segunda parte de este Laboratorio 8 necesitaremos TensorFlow y TensorFlow-Datasets. Además, como habitualmente, fijaremos la semilla aleatoria para asegurar la reproducibilidad de los experimentos.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Fijamos la semilla para poder reproducir los resultados
import os
import numpy as np
import random
seed=1234
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

2024-11-08 11:31:34.110557: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 11:31:34.114776: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-08 11:31:34.133223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731061894.166052   10489 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731061894.174011   10489 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 11:31:34.201177: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Además, cargamos también APIs que vamos a emplear para que el código quede más legible

In [2]:
#API de Keras, modelo Sequential y la capa Dense 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense 
#Para mostrar gráficas
from matplotlib import pyplot

### Carga del conjunto de datos

De nuevo, seguimos empleando el conjunto *german_credit_numeric* ya empleado en los laboratorios anteriores.

In [3]:
# TODO: Carga el conjunto german_credit como ds_train
# Indica además un tamaño de batch de 128 y que se repita indefinidamente
ds_train = tfds.load('german_credit_numeric', split='train',as_supervised=True).batch(128).repeat()

W0000 00:00:1731061916.771884   10489 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Visualización del desvanecimiento del gradiente
En este apartado visualizaremos los tamaños de los gradientes, como hicimos en la primera parte. Para ello mantenemos la declaración de `GradientLoggingSequentialModel`.

In [4]:
class GradientLoggingSequentialModel(tf.keras.models.Sequential):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # En la inicialización instanciamos una nueva variable en la que
        # registraremos el historial de tamaños de gradientes de cada capa
        self.gradient_history = {}
    
    def compile(self, **kwargs):
        result = super().compile(**kwargs)
        # Una vez sabemos la arquitectura, podemos inicializar la historia
        # de gradientes de cada capa a una lista vacía.
        for l in self.layers:
            self.gradient_history[l.name] = []
        return result
        
    def _save_gradients(self, gradients):
        # A cada paso de entrenamiento llamaremos a esta función para que
        # registre los gradientes.
        # En la lista gradients se encuentran los gradientes de las distintas
        # capas por orden. Cada capa l tendrá un número de gradientes que
        # concidirá con l.trainable_variables.
        # Teniendo esto en cuenta, recorremos los gradientes, calculamos su
        # tamaño y guardamos la media de tamaños de cada capa en el histórico
        i = 0
        for layer in self.layers:
            gradient_sizes = []
            for lw in layer.trainable_variables:
                g_size = np.linalg.norm(gradients[i].numpy())
                gradient_sizes.append(g_size)
                i += 1
            mean_gradient_size = np.mean(gradient_sizes)
            self.gradient_history[layer.name].append(mean_gradient_size)
        
    def train_step(self, data):
        # Haremos un paso de entrenamiento personalizado basado en 
        # https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit#a_first_simple_example
        # Dejaremos el ejemplo tal cual, añadiendo tan solo la llamada a
        # _save_gradients una vez que disponemos de los gradientes
        
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        
        # Llamada añadida para grabar los gradientes.
        self._save_gradients(gradients)
        
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

### Creación del bloque con conexión residual

En una red neuronal *feed-forward* convencional, la salida de cada capa se utiliza como entrada de las capa siguiente.
<img src="./img/resnet.png" alt="Dos capas en una red FF" width="400"/>

En contraste, en una ResNet se introducen bloques que incluyen conexiones residuales con el objetivo de favorecer la propagación de los gradientes.
<img src="./img/resnet-2.png" alt="Dos capas en una red FF con conexión residual" width="400"/>

A pesar de que las ResNet se suelen utilizar con redes convolucionales, en este Laboratorio utilizaremos una red *feed-forward*.

Para poder utilizar este tipo de bloques en nuestra arquitectura definiremos un nuevo tipo de modelo denominado `DoubleDenseWithSkipModel` que consistirá en lo representado en la imagen:
 - Una primera capa Dense, cuya salida sirve de entrada a...
 - Una segunda capa Dense lineal (sin activación), cuya salida sirve de entrada a...
 - A una operación suma que la añade a la entrada original a la primera capa. La salida de esta suma servirá de entrada a...
 - Una función de activación, cuya salida será la salida del bloque.

Utilizaremos la función sigmoide como función de activación en ambos casos. La entrada y la salida del bloque deberán tener la misma dimensión para que se pueda realizar la suma. Por simplicidad, mantendremos la salida de la primera capa con la misma dimensión.

Nuestra nueva clase heredará de `Model`, por lo que deberá implementar los métodos `build` y `call`. En celdas posteriores añadiremos estos bloques a un modelo `Sequential` como si fuesen capas.

In [ ]:
class DoubleDenseWithSkipModel(tf.keras.models.Model):
    def __init__(self, **kwargs):
        super(DoubleDenseWithSkipModel, self).__init__(**kwargs)

    # TODO: Completa el método build
    def build(self, input_shape):
        ...
        
    # TODO: Completa el método skip
    def call(self, x):
        ...

## Creamos un modelo *GradientLoggingSequentialModel* utilizando las nuevas capas
Creamos un modelo *GradientLoggingSequentialModel* para ajustar a los datos de entrada siguiendo las especificaciones dadas. Deberá incluir (además de las capas de entrada y salida) una capa de 10 unidades con activación sigmoide y 10 de los nuevos bloques.

In [ ]:
# TODO - Define en model una red GradientLoggingSequentialModel
# Pon una capa densa con 10 unidades con activación sigmoide y 10 capas DoubleDenseWithSkipModel
model = ...

#Construimos el modelo y mostramos 
model.build()
print(model.summary())

### Entrenamiento del modelo
Como en la primera parte, vamos a establecer la función de pérdida (entropía cruzada binaria), el optimizador (SGD con LR $10^{-3}$) y la métrica que nos servirá para evaluar el rendimiento del modelo entrenado (área bajo la curva).

In [ ]:
#TODO - Compila el modelo. Utiliza la opción run_eagerly=True para que se puedan registrar los gradientes a cada paso
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Entrenamos el modelo usando model.fit

In [ ]:
#TODO - entrenar el modelo utilizando 8 steps por epoch. Con 10 epochs nos valdrá para comprobar el desvanecimiento de gradientes.
history = model.fit(ds_train, epochs=10, steps_per_epoch=8)


Una vez hayas encontrado un valor de learning rate que consiga una convergencia rápida, guarda el history de la pérdida en la variable history_sgd para poder hacer comparativas.

In [ ]:
pyplot.figure(figsize=(14, 6), dpi=80)
pyplot.boxplot(model.gradient_history.values())
pyplot.yscale('log')
pyplot.xticks(ticks=range(1,len(model.gradient_history)+1), labels=model.gradient_history.keys())
pyplot.show()

## Comparativa
 - Contrasta los resultados con los obtenidos en la primera parte
 - Cambia las activaciones del bloque y de la primera capa oculta de la red a ReLU y observa la diferencia.
 - Alarga el entrenamiento y prueba distintos optimizadores para intentar que el modelo entrene correct